In [1]:
import torch
import yolov5.utils
display = yolov5.utils.notebook_init()  # checks

%matplotlib inline

YOLOv5 🚀 2024-1-24 Python-3.8.10 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 7931MiB)


Setup complete ✅ (16 CPUs, 14.8 GB RAM, 184.7/245.0 GB disk)


In [2]:
class RGBD():
    def __init__(self):
        
        self._cloud_sub = rospy.Subscriber(
            "/camera/depth_registered/points",
            PointCloud2, self._cloud_cb)
        self._points_data = None
        self._image_data = None
        

    def _cloud_cb(self, msg):
        self._points_data = ros_numpy.numpify(msg)
        self._image_data = self._points_data['rgb'].view((np.uint8, 4))[..., [2, 1, 0]]
        

    def get_image(self):
        return self._image_data

    def get_points(self):
        return self._points_data

In [3]:
import tf
import rospy
from sensor_msgs.msg import Image , LaserScan , PointCloud2
rgbd = RGBD()


In [9]:
import matplotlib.pyplot as plt
import cv2

test_img=rgbd.get_image()
test_img= cv2.cvtColor(test_img, cv2.COLOR_RGB2BGR)## CONVERT TO CV U MAT
test_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
plt.imshow(test_img)

error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
